# Connect to openrouter.ai using openai API and use free LLM models

## Imports

In [15]:
import os
from dotenv import load_dotenv
from openai import OpenAI

from IPython.display import Markdown, display, update_display

## app setup

In [7]:
load_dotenv()

openrouter_base_url = os.getenv("OPENROUTER_BASE_URL")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

openrouter_llm_model = os.getenv("MINIMAX_FREE")

if (openrouter_base_url and openrouter_api_key and openrouter_llm_model) is None:
    print('Openrouter Base url and api key are missing')
else:
    print(f"""Found base url and api key.
          Base URL: {openrouter_base_url}
          API Key: {openrouter_api_key[:10]}
          LLM Model: {openrouter_llm_model}""")
    


Found base url and api key.
          Base URL: https://openrouter.ai/api/v1
          API Key: sk-or-v1-b
          LLM Model: minimax/minimax-m2:free


## Connect to LLM model


In [14]:
client = OpenAI(base_url=openrouter_base_url, api_key=openrouter_api_key)

payload = [
    {'role': 'system', 'content': 'You are a humorous assisant.'},
    {'role': 'user', 'content': 'Tell me a fun fact about lions.'}
]

In [ ]:

response = client.chat.completions.create(model=openrouter_llm_model, messages=payload)

print(response.choices[0].message.content)

Fun fact: Lions are the only cats that live in social groups called “prides,” and their roar can be heard up to 5 miles (8 km) away across the African savannah!


Write another prompt and stream the output

In [16]:
payload = [
    {'role': 'system', 'content': 'You are a expert software engineer, having vast experience in DSA ALGO, AI ML, using python, java and Springboot tech stack.'},
    {'role': 'user', 'content': 'Write an algorithm with example for greedy algorithm and explain it for begineers.'}
]

stream_resp = client.chat.completions.create(model=openrouter_llm_model, messages=payload, stream=True)

display_writer = display(Markdown(""), display_id=True)
resp = ""
for chunk in stream_resp:
    resp += chunk.choices[0].delta.content
    update_display(Markdown(resp), display_id=display_writer.display_id)


### Greedy Algorithm: An Introduction with the Coin Change Problem

The **greedy algorithm** is a simple, intuitive approach for solving optimization problems. It works by making the **locally optimal choice** at each step, hoping that these choices lead to a globally optimal solution.  
Key idea: "At each step, pick the best option you see right now."  
**Important**: Greedy works *only* if the problem has the **optimal substructure** property (optimal solution contains optimal solutions to subproblems) and **no need for backtracking**.

---

### Example Problem: Coin Change
**Goal**: Given coin denominations (e.g., `[1, 5, 10, 25]`) and a target amount (e.g., `63`), find the **minimum number of coins** needed to make that amount.

#### Step-by-Step Greedy Approach:
1. **Sort coins in descending order** (largest first).  
2. **Iterate from largest to smallest coin**:  
   - Use as many of the current coin as possible without exceeding the target.  
3. **Update the remaining amount** after each step.  
4. **Repeat** until the remaining amount is `0`.

---

#### Example:  
- **Denominations**: `[1, 5, 10, 25]`  
- **Target**: `63`

| Step | Coin Chosen | Count Used | Remaining Amount |
|------|-------------|------------|------------------|
| 1    | 25          | 2 (50)     | 63 - 50 = 13     |
| 2    | 10          | 1 (10)     | 13 - 10 = 3      |
| 3    | 5           | 0          | 3                |
| 4    | 1           | 3          | 3 - 3 = 0        |

**Result**: Minimum coins = `2 (25) + 1 (10) + 3 (1) = 6 coins`.

---

### Algorithm (Python Code):
```python
def greedy_coin_change(coins, target):
    coins.sort(reverse=True)  # Sort in descending order
    count = 0
    
    for coin in coins:
        if target == 0:
            break
        # Use as many of this coin as possible
        num_coins = target // coin
        count += num_coins
        target -= num_coins * coin  # Update remaining amount
    
    return count

# Example usage:
coins = [1, 5, 10, 25]
target = 63
print(greedy_coin_change(coins, target))  # Output: 6
```

---

### Why Does Greedy Work Here?
- **Coin denominations are canonical** (like US coins). Using larger coins first minimizes the total coins.  
- **No backtracking needed**: Each step’s choice is final (unlike problems like the [Knapsack problem](https://www.geeksforgeeks.org/greedy-algorithm-2/)).

### When Greedy Fails:
If denominations were `[1, 3, 4]` and target=`6`:  
- Greedy picks `4 + 1 + 1 = 3 coins` (bad).  
- Optimal: `3 + 3 = 2 coins`.  

Greedy fails because "locally best" choices (4) don’t lead to a global optimum.

---

### Key Takeaways for Beginners:
1. **Greedy = Repeatedly pick the "seemingly best" option**.  
2. **Works only if** the problem allows "no regrets" (e.g., no need to reconsider past choices).  
3. **Not universal**: Use dynamic programming if greedy fails.  
4. **Example problems**: Coin change, fractional knapsack, activity selection, Dijkstra’s algorithm.  

> **Pro Tip**: Always verify greedy solutions against brute-force results for small cases to ensure correctness!